In [3]:
import pandas as pd
import numpy as np
from datetime import datetime

In [49]:
data = pd.read_csv('sleep_awake.csv')
data.head()

,Date,Sleep,Wake,Unnamed: 3
0,12/1/2017,16,1,NaN
1,12/2/2017,19,-10,NaN
2,12/3/2017,-45,0,NaN
3,12/4/2017,8,1,NaN
4,12/5/2017,-23,-16,NaN


In [50]:
data = data.ix[:, 0:3]
data.head()

C:\Users\sidha\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


,Date,Sleep,Wake
0,12/1/2017,16,1
1,12/2/2017,19,-10
2,12/3/2017,-45,0
3,12/4/2017,8,1
4,12/5/2017,-23,-16


In [53]:
data = data.ix[:, 0:3]

data['Sleep_time'] = [datetime.strptime(entry[0], '%m/%d/%Y') + pd.DateOffset(minutes=entry[1], hours=22) for entry in zip(data['Date'], data['Sleep'])]
data['Wake_time'] = [datetime.strptime(entry[0], '%m/%d/%Y') + pd.DateOffset(minutes=entry[1], hours=6) for entry in zip(data['Date'], data['Wake'])]
data.head()

C:\Users\sidha\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


,Date,Sleep,Wake,Sleep_time,Wake_time
0,12/1/2017,16,1,2017-12-01 22:16:00,2017-12-01 06:01:00
1,12/2/2017,19,-10,2017-12-02 22:19:00,2017-12-02 05:50:00
2,12/3/2017,-45,0,2017-12-03 21:15:00,2017-12-03 06:00:00
3,12/4/2017,8,1,2017-12-04 22:08:00,2017-12-04 06:01:00
4,12/5/2017,-23,-16,2017-12-05 21:37:00,2017-12-05 05:44:00


In [54]:
pd.date_range?

In [58]:
# aim is to add the date time before and after the given time
sleep_index = pd.date_range(start = min(data.Date), end = max(data.Date), freq = '60S')

In [65]:
sleep_index = [time for time in sleep_index if(time.hour >= 21) & (time.hour <= 23)]

In [75]:
sleep_data = pd.DataFrame(0, columns=['indicator'], index=sleep_index)
sleep_data.head()

,indicator
2018-01-01 21:00:00,0
2018-01-01 21:01:00,0
2018-01-01 21:02:00,0
2018-01-01 21:03:00,0
2018-01-01 21:04:00,0


In [84]:
for time in sleep_data.index:
    for sleep_time in data['Sleep_time']:
        if time == sleep_time:
            #print(time)
            sleep_subset = [date for date in sleep_data.index if date.date() == time.date()]
            asleep_subset = [time for time in sleep_subset if time.time() >= sleep_time.time()]
            sleep_data.loc[asleep_subset,'indicator'] = 1

In [95]:
sleep_data.indicator.value_counts()

1    3230
0    2530
Name: indicator, dtype: int64

In [96]:
# Create a separate wake dataframe
wake_index = pd.date_range(start=min(data.Date), end=max(data.Date), freq='60S')
wake_index = [time for time in wake_index if (time.hour <= 7) & (time.hour >= 5)]
wake_data = pd.DataFrame(1, columns=['indicator'], index=wake_index)

# Set an index with indicator for times when I am asleep
# Indicator = 1 when I am asleep, else 0
for time in wake_data.index:
    for wake_time in data['Wake_time']:
        if time == wake_time:
            selected_date = time.date()
            date_subset = [date for date in wake_data.index if date.date() == selected_date]
            wake_times = [time for time in date_subset if time.time() >= wake_time.time()]
            wake_data.loc[wake_times, 'indicator'] = 0
    

C:\Users\sidha\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  


In [102]:
sleep_data.head()

,indicator,time_offset
2018-01-01 21:00:00,0,-60
2018-01-01 21:01:00,0,-59
2018-01-01 21:02:00,0,-58
2018-01-01 21:03:00,0,-57
2018-01-01 21:04:00,0,-56


In [99]:
sleep_data['time_offset'] = [int((time.hour * 60 + time.minute) - (22 * 60)) for time in sleep_data.index]
wake_data['time_offset'] = [int((time.hour * 60 + time.minute) - (6 * 60)) for time in wake_data.index]

In [101]:
sleep_data.to_csv('sleep_data.csv')
wake_data.to_csv('wake_data.csv')